# Sympy derivation of matrix elements

In [ ]:
import sympy as sp

Define some symbols

In [ ]:
theta_w, phi_t, PA, inc, azi = sp.symbols('theta_w, phi_t, PA_, inc_, azi_', real=True)
x, y, z = sp.symbols('x, y, z', real=True)

Define the rotation matrices.

In [ ]:
def Rx(angle):
    return sp.Matrix([
    [1,       0,              0],
    [0,       sp.cos(angle), -sp.sin(angle)],
    [0,       sp.sin(angle),  sp.cos(angle)],
])

def Ry(angle):
    return sp.Matrix([
    [ sp.cos(angle), 0, sp.sin(angle)],
    [0,              1, 0],
    [-sp.sin(angle), 0, sp.cos(angle)],
])

def Rz(angle):
    return sp.Matrix([
    [sp.cos(angle), -sp.sin(angle), 0],
    [sp.sin(angle),  sp.cos(angle), 0],
    [0,              0,             1],
])

Define the initial coordinates $\vec p_0 = \begin{pmatrix}
x\\
y\\
z
\end{pmatrix}$

In [ ]:
p0 = sp.Matrix([x, y, z])

We apply the tilting and twisting to the disk coordinates which is a rotation first around $\vec x$ and then around $\vec z$

In [ ]:
p1 = Rz(phi_t) * Rx(theta_w) * p0

Next we get the observers view by projecting onto a new coordinate axis that is rotated by `PA`. If `PA` is zero, the observer is looking down onto the disk in negative z direction.

In [ ]:
# unrotated base: top view
base0 = sp.Matrix([[1, 0, 0],[0, 1, 0], [0, 0, -1]])

# inclination of the entire disk
R_inc = Rx(inc)

# azimuthal rotation of the view
R_azi = Ry(azi)

# rotate the base
R_PA = Rz(PA)
base1 = R_PA * base0

now project onto that new base

In [ ]:
p2 = ((R_azi * R_inc * p1).T * base1).T

The result is rather messy ...

In [ ]:
p2

... so we rather let `sympy` write the fortran code for us. This is what is put into the `fortran.f90` file.

In [ ]:
from sympy.printing import fcode
print(fcode(p2[0], standard=2003, source_format='free'))
print('')
print(fcode(p2[1], standard=2003, source_format='free'))
print('')
print(fcode(p2[2], standard=2003, source_format='free'))